In [1]:
import numpy as np
import tensorflow as tf
import edward as ed

## Models in Edward

* Models in Edward are inductively constructed as a static computation graph
    * Each node can be a random variable, a deterministic computation step, or a neural network
* Models can be composed by extending the graph at relevant nodes

## Primitive stochastic functions

* Use **Edward**'s models
* Can create custom distributions using transforms or by inheriting directly from `RandomVariable`

### Drawing samples from unit Normal

In [2]:
loc = 0.   # mean zero
scale = 1. # unit variance

In [3]:
normal = ed.models.Normal(loc, scale) # creates a normal distribution object

In [4]:
x = normal.sample() # draws a sample from N(0, 1)

with tf.Session() as sess:
    vx, vlp = sess.run([x, normal.log_prob(x)])
    print("sample", vx)
    print("log prob", vlp)

sample 0.162869
log prob -0.9322017


### Sampling

* Samples are named hierarchically

In [5]:
x = normal.sample(name="x")

with tf.Session() as sess:
    vx = sess.run(x)
    print("sample", x, vx)

sample Tensor("Normal/x/Reshape:0", shape=(), dtype=float32) 0.8472964


## Simple model

In [6]:
def weather():
    cloudy = ed.models.Bernoulli(0.3).sample(name="cloudy")
    cloudy = tf.cond(tf.equal(cloudy, 1), lambda: tf.constant('cloudy'), lambda: tf.constant('sunny'))
    mean_temp = tf.cond(tf.equal(cloudy, 'cloudy'), lambda: tf.constant(55.0), lambda: tf.constant(75.0))
    scale_temp = tf.cond(tf.equal(cloudy, 'cloudy'), lambda: tf.constant(10.0), lambda: tf.constant(15.0))
    temp = ed.models.Normal(mean_temp, scale_temp).sample(name='temp')
    return cloudy, temp

with tf.Session() as sess:
    (cloudy, temp) = weather()
    for _ in range(3):
        print(sess.run((cloudy, temp)))

(b'cloudy', 38.88389)
(b'sunny', 82.539665)
(b'cloudy', 60.88809)


In [7]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = tf.cond(tf.equal(cloudy, "sunny"), lambda: tf.constant(200.0), lambda: tf.constant(50.0))
    ice_cream = ed.models.Normal(expected_sales, 10.0).sample(name='ice_scream')
    return cloudy, temp, ice_cream

with tf.Session() as sess:
    print(sess.run(ice_cream_sales()))

(b'cloudy', 63.0123, 45.523544)


## Stochastic Recursion, Higher-order Stochastic Functions, and Random Control Flow 

* **Edward** builds on **TensorFlow** and thus does not support functions nor general recursion.
* Instead, it supports a restricted form of `while` loops (through `tf.while_loop`).
* However, many recursive functions can still be converted into usages of `tf.while_loop`.

In [8]:
def geometric(p):
    return tf.while_loop(
        cond=lambda _: tf.equal(ed.models.Bernoulli(probs=p).sample(), 0), # name is automatically generated
        body=lambda t: tf.add(t, 1),
        loop_vars=[tf.constant(0)],
    )

with tf.Session() as sess:
    acc = sess.run(geometric(0.001))
    print(acc)

390


## Inference

In [9]:
def scale(guess):
    var = 1.0
    # The prior over weight encodes our uncertainty about our guess
    weight = ed.models.Normal(guess, var)
    # This encodes our belief about the noisiness of the scale:
    # the measurement fluctuates around the true weight
    measurement = ed.models.Normal(weight, 0.75)
    return weight, measurement

def experiment():
    weight, measurement = scale(8.5)
    posterior = ed.models.Empirical(tf.Variable(tf.zeros(10000)))
    inference = ed.HMC({weight: posterior}, {measurement: 9.5})
    tf.global_variables_initializer().run()
    inference.run()
    vmean, vstd = ed.get_session().run((posterior.mean(), posterior.stddev()))
    print("mean", vmean)
    print("stddev", vstd)

experiment()

10000/10000 [100%] ██████████████████████████████ Elapsed: 8s | Acceptance Rate: 0.988
mean 9.120355
stddev 0.6143609


## Variational Inference

In [10]:
def guide(guess):
    qalpha = tf.Variable(ed.models.Normal(0.0, 1.0).sample() + guess, name='a')
    qbeta = tf.Variable(ed.models.Normal(0.0, 1.0).sample(), name='b')
    qweight = ed.models.Normal(qalpha, qbeta)
    return qalpha, qbeta, qweight

def vi_experiment():
    guess = tf.Variable(8.5)
    weight, measurement = scale(guess)
    qalpha, qbeta, qweight = guide(guess)
    inference = ed.KLqp({weight: qweight}, {measurement: 9.5})
    tf.global_variables_initializer().run()
    inference.run(n_samples=10, n_iter=10000)
    vguess, vmean, vstd = ed.get_session().run((guess, qalpha, qbeta))
    print('guess', vguess)
    print("mean", vmean)
    print("stddev", vstd)

vi_experiment()

10000/10000 [100%] ██████████████████████████████ Elapsed: 9s | Loss: 1.182
guess 9.500665
mean 9.500643
stddev 0.5976114


## Variational Auto Encoder

In [11]:
mnist = tf.keras.datasets.mnist

def generator(array, batch_size):
    """Generate batch with respect to array's first axis."""
    start = 0  # pointer to where we are in iteration
    while True:
        stop = start + batch_size
        diff = stop - array.shape[0]
        if diff <= 0:
            batch = array[start:stop]
            start += batch_size
        else:
            batch = np.concatenate((array[start:], array[:diff]))
            start = diff
        batch = batch.astype(np.float32) / 255.0  # normalize pixel intensities
        batch = np.random.binomial(1, batch)  # binarize images
        yield batch.reshape(-1, 784) # shape the mini-batch to have pixels in the rightmost dimension


def vae(nepoch, batch_size, latent_dimension, learning_rate):
    (x_train, _),(x_test, _) = mnist.load_data()
    x_train_generator = generator(x_train, batch_size)
    # decoder
    z = ed.models.Normal(tf.zeros([batch_size, latent_dimension]), tf.ones([batch_size, latent_dimension]))
    hidden = tf.layers.dense(z, 256, activation=tf.nn.relu)
    x = ed.models.Bernoulli(logits=tf.layers.dense(hidden, 28 * 28))
    # encoder
    x_ph = tf.placeholder(tf.int32, [batch_size, 28 * 28])
    hidden = tf.layers.dense(tf.cast(x_ph, tf.float32), 256, activation=tf.nn.relu)
    qz = ed.models.Normal(tf.layers.dense(hidden, latent_dimension),
                tf.layers.dense(hidden, latent_dimension, activation=tf.nn.softplus))
    # bind decoder and encoder
    inference = ed.KLqp({z: qz}, data={x: x_ph})
    optimizer = tf.train.AdamOptimizer(learning_rate)
    inference.initialize(optimizer=optimizer)
    tf.global_variables_initializer().run()

    n_iter_per_epoch = x_train.shape[0] // batch_size
    for epoch in range(1, nepoch + 1):
        print("Epoch: {0}".format(epoch))
        avg_loss = 0.0

        pbar = ed.util.Progbar(n_iter_per_epoch)
        for t in range(1, n_iter_per_epoch + 1):
            pbar.update(t)
            x_batch = next(x_train_generator)
            info_dict = inference.update(feed_dict={x_ph: x_batch})
            avg_loss += info_dict['loss']

        # print a lower bound to the average marginal likelihood for an image
        avg_loss /= n_iter_per_epoch
        avg_loss /= batch_size
        print("-log p(x) <= {:0.3f}".format(avg_loss))

vae(100, 256, 400, 1.0e-3)

Epoch: 1
234/234 [100%] ██████████████████████████████ Elapsed: 10s
-log p(x) <= 206.073
Epoch: 2
234/234 [100%] ██████████████████████████████ Elapsed: 10s
-log p(x) <= 154.291
Epoch: 3
234/234 [100%] ██████████████████████████████ Elapsed: 10s
-log p(x) <= 144.893
Epoch: 4
234/234 [100%] ██████████████████████████████ Elapsed: 9s
-log p(x) <= 139.864
Epoch: 5
234/234 [100%] ██████████████████████████████ Elapsed: 11s
-log p(x) <= 135.654
Epoch: 6
234/234 [100%] ██████████████████████████████ Elapsed: 10s
-log p(x) <= 131.181
Epoch: 7
234/234 [100%] ██████████████████████████████ Elapsed: 10s
-log p(x) <= 126.816
Epoch: 8
234/234 [100%] ██████████████████████████████ Elapsed: 9s
-log p(x) <= 122.847
Epoch: 9
234/234 [100%] ██████████████████████████████ Elapsed: 10s
-log p(x) <= 119.230
Epoch: 10
234/234 [100%] ██████████████████████████████ Elapsed: 10s
-log p(x) <= 116.200
Epoch: 11
234/234 [100%] ██████████████████████████████ Elapsed: 10s
-log p(x) <= 113.931
Epoch: 12
234/234 [10